# Automated interaction statistics

This notebooks aims to provide an automated pipeline to generate **protein-ligand interaction statistics** for any protein with available protein:ligand complexes in the PDB. The only required parameter is the **UniProt ID** of the protein of interest.

## Content
* Data collection
    * Collect relevant pdb codes
    * Collect ligands bound to PDB structures
    * Download and prepare runumbered structures
* Data processing
    * Load structures
    * Identify common residues
    * Generate pocket mappings
* Interaction statistics
    * Generate interaction fingerprint
    * Apply KLIFS rediue numbering (if applicable) 

In [1]:
import json
from pathlib import Path
import urllib
import zlib

import biotite.database.rcsb as rcsb
import requests
from tqdm.auto import tqdm

from plipify.core import Structure
from plipify.fingerprints import InteractionFingerprint

Beside the [UniProt](https://www.uniprot.org/) ID, one can further customize the PDB query for potential protein:ligand complexes, i.e. 
* number of mutations, 
* experimental method, 
* ligand type, 
* and the molecular weight of the co-crystallized ligand(s)

The pipeline is exemplified for the protein kinase **ABL1** ([P00519](https://www.uniprot.org/uniprot/P00519)).

In [2]:
uniprot_id = "P00519"  # ABL1
#uniprot_id = "P00533"  # EGFR
#uniprot_id = "Q9NRG4"  # SMYD2
max_mutation_count = 0
experimental_method = "X-RAY DIFFRACTION"
ligand_type = "non-polymer"
ligand_mw = (300, 500)

Create data folder (if it does not exist yet)

In [3]:
HERE = Path(_dh[-1])
DATA = HERE / "data" / uniprot_id
DATA.mkdir(exist_ok=True, parents=True)

# Data collection

## Collect relevant pdb codes

In the next step, biotite's [rcsb module](https://www.biotite-python.org/apidoc/biotite.database.rcsb.html#module-biotite.database.rcsb) will be used to query the PDB for relevant structures. The query is thereby customized by the parameters given above.
* Collect individual query strings
* Combine them and query PDB

In [4]:
query_by_uniprot = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_name",
    exact_match="UniProt"
)
query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id
)
query_mutation_count = rcsb.FieldQuery(
    "entity_poly.rcsb_mutation_count",
    range_closed=(0, max_mutation_count)
)
query_by_experimental_method = rcsb.FieldQuery(
    "exptl.method",
    exact_match=experimental_method
)
query_by_ligand_type = rcsb.FieldQuery(
    "chem_comp.type",
    exact_match=ligand_type
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight",
    range_closed=ligand_mw
)

Compose query.

In [5]:
results = rcsb.search(
    rcsb.CompositeQuery(
        [
            query_by_uniprot,
            query_by_uniprot_id,
            query_mutation_count,
            query_by_experimental_method,
            query_by_ligand_type,
            query_by_ligand_mw
        ],
        operator="and"
    ),
    return_type="polymer_instance"
)

Collect matching pdb

In [6]:
pdbs = []
for result in results:
    pdb_id, chain = result.split(".")
    pdbs.append({"pdb_id": pdb_id, "chain": chain})

print(f"Found {len(pdbs)} structures in the PDB.")

Found 30 structures in the PDB.


## Collect ligands bound to PDB structures

Currently, it seems not possible to query the PDB about co-crystallized ligands in a programmatic fashion. Hence, we will perform manual requests and directly filter for ligands with a matching molecular weight.

In [7]:
base_url = "https://data.rcsb.org/graphql?query="
potential_complexes = []
for pdb in tqdm(pdbs):
    pdb_id = pdb["pdb_id"]
    query = '{entry(entry_id:"' + pdb_id + '"){nonpolymer_entities{nonpolymer_comp{chem_comp{id,formula_weight}}}}}'
    response = requests.get(base_url + urllib.parse.quote(query))
    potential_ligands = json.loads(response.text)["data"]["entry"]["nonpolymer_entities"]
    # remove ligands without molecular weight
    potential_ligands = [
        ligand
        for ligand in potential_ligands 
        if ligand["nonpolymer_comp"]["chem_comp"]["formula_weight"] is not None
    ]
    # filter for molecular weight
    ligands = [
        ligand["nonpolymer_comp"]["chem_comp"]["id"]
        for ligand in potential_ligands 
        if ligand_mw[0] <= ligand["nonpolymer_comp"]["chem_comp"]["formula_weight"] <= ligand_mw[1]
    ]
    # collect chain:ligand pairs
    for ligand in ligands:
        potential_complexes.append(
            {"pdb_id": pdb_id, 
             "chain": pdb["chain"],
             "ligand": ligand}
        )
print(f"Identified {len(potential_complexes)} potential protein[chain]:ligand complexes.")

  0%|          | 0/30 [00:00<?, ?it/s]

Identified 44 potential protein[chain]:ligand complexes.


## Download and prepare renumbered structures

Structures of the same protein deposited in the PDB are not guaranteed to have the same residue numbering. However, this is important for plipify to compare interactions between different structures. Luckily, the Dunbrack Lab provides the [PDBrenum](http://dunbrack3.fccc.edu/PDBrenum/) service that allows to renumber structures according to the UniProt ID. The following code will download the respective structures and prepare separate file for each chain:ligand combination.

In [8]:
valid_complexes = []
for potential_complex in tqdm(potential_complexes):
    pdb_id = potential_complex["pdb_id"]
    chain = potential_complex["chain"]
    ligand = potential_complex["ligand"]
    pdb_path = DATA / f"{pdb_id}_{chain}_{ligand}.pdb"
    response = requests.get(f"http://dunbrack3.fccc.edu/PDBrenum/output_PDB/{pdb_id.lower()}_renum.pdb.gz")
    ligand_identified = False
    with open(pdb_path, "w") as pdb_file:
        for line in zlib.decompress(response.content, 15+32).decode("utf-8").split("\n"):
            if line.startswith(("ATOM", "HETATM", "ANISOU", "TER")):
                if line[21] == chain:  # filter for chain
                    if line.startswith("HETATM"):
                        if line[17:20] == ligand:  # filter for ligand, might be problematic for co-factors
                            ligand_identified = True
                            pdb_file.write(line + "\n")
                    else:
                        pdb_file.write(line + "\n")
            else:
                pdb_file.write(line + "\n")
    # remove structure without accompanying ligand
    if not ligand_identified:
        pdb_path.unlink()
    else:
        potential_complex["file_path"] = pdb_path
        valid_complexes.append(potential_complex)

print(f"Generated separate PDB files for {len(valid_complexes)} protein[chain]:ligand complexes.")

  0%|          | 0/44 [00:00<?, ?it/s]

Generated separate PDB files for 40 protein[chain]:ligand complexes.


# Data processing

## Load structures

Next, all structures will be loaded into plipify and checked for the existence of multiple ligands. This can happen if the same ligand is present twice in the same chain, e.g. 5V3H.

In [9]:
valid_structures = []
for valid_complex in tqdm(valid_complexes):
    structure = Structure.from_pdbfile(str(valid_complex["file_path"]))
    if len(structure.binding_sites) != 1:
        print(f"{valid_complex['pdb_id']} chain {valid_complex['chain']} contains {len(structure.binding_sites)} binding sites but we want exactly one.")
    else:
        valid_structures.append(structure)
print(f"Generated {len(valid_structures)} valid protein[chain]:ligand complexes.")

  0%|          | 0/40 [00:00<?, ?it/s]

Generated 40 valid protein[chain]:ligand complexes.


## Identify common residues

Additionally, we will restrict the interaction analysis to the residues that are common to all structures.

In [10]:
resid_sets = []
for structure in valid_structures:
    protein_resids = [residue.seq_index for residue in structure.residues if residue.is_protein()]
    resid_sets.append(set(protein_resids))

common_resids = resid_sets[0].intersection(*resid_sets)
print(f"Identified {len(common_resids)} residues common to all structures.")

Identified 202 residues common to all structures.


## Generate pocket mappings

These common residues can be passed as pocket mappings to plipify to generate the interaction statistics.

In [11]:
pocket_mappings = []
for structure in valid_structures:
    chain = structure.identifier.split("_")[1]
    mapping = {}
    for common_resid in common_resids:
        mapping[common_resid] = {
            "seq_index": common_resid,
            "chain": chain,
        }
    pocket_mappings.append(mapping)

# Interaction statistics

## Generate interaction fingerprint

Finally, the interaction fingerprint will be generated.

In [12]:
fp = InteractionFingerprint().calculate_fingerprint(
        valid_structures,
        residue_indices=pocket_mappings,
        labeled=True, 
        as_dataframe=True, 
        remove_non_interacting_residues=True,
        remove_empty_interaction_types=True,
        ensure_same_sequence=False
    )

if not fp.values.shape[0]:
    raise ValueError("Fingerprint is empty!")

Show the dataframe with interactions colored by frequency.

In [13]:
fp.style.background_gradient(axis=None, cmap="YlGnBu")

,hydrophobic,hbond-don,hbond-acc,saltbridge,pistacking,pication,halogen
248,26,0,0,0,0,0,0
256,12,0,0,0,0,0,0
261,2,0,0,0,0,0,0
264,4,0,1,0,1,0,0
269,13,0,0,0,0,0,2
271,13,1,0,0,0,3,0
293,2,0,0,0,0,0,0
299,10,0,0,0,0,0,0
302,1,0,0,0,0,0,0
313,21,0,0,0,0,0,0


## Apply KLIFS residue numbering (if applicable)
In this example, we chose a kinase example, thus, we decided to convert the residue numbers to the [KLIFS](https://klifs.net/) numbering scheme.

Note:
* If you are workin with a different protein, please skip these steps. 
* Or if you work with another protein family for which such a numbering scheme is available, it could be integrated as well. 

In [14]:
from opencadd.databases.klifs import setup_remote

In [15]:
# Set up remote session
klifs = setup_remote()

Note: `klifs_kinase_ID` for the kinase was checked manually. 

In [16]:
klifs_kinase_id = 1061  # ABL1 4WA9
pocket = klifs.pockets.by_structure_klifs_id(klifs_kinase_id)

Reindex the interactions statistics dataframe.

In [17]:
for index in fp.index:
    try:
        klifs_resid = int(pocket[pocket["residue.id"] == str(index)]["residue.klifs_id"].iloc[0])
    except IndexError:
        continue
    fp.rename(index={index: f"KLI{klifs_resid:02d}"}, inplace=True)

Show the dataframe with interactions colored by frequency.

In [18]:
fp.style.background_gradient(axis=None, cmap="YlGnBu")

,hydrophobic,hbond-don,hbond-acc,saltbridge,pistacking,pication,halogen
KLI03,26,0,0,0,0,0,0
KLI11,12,0,0,0,0,0,0
261,2,0,0,0,0,0,0
264,4,0,1,0,1,0,0
KLI15,13,0,0,0,0,0,2
KLI17,13,1,0,0,0,3,0
KLI31,2,0,0,0,0,0,0
KLI36,10,0,0,0,0,0,0
KLI39,1,0,0,0,0,0,0
KLI43,21,0,0,0,0,0,0
